In [1]:
import theano as th
import theano.tensor as T
from theano import function

In [2]:
import numpy as np

In [3]:
x = T.dvector()
y = T.dscalar()
z = x*y  
f = function([x, y], z)

In [ ]:
f(np.ones(10000),2)

In [ ]:
output, updates = th.scan(fn=lambda x, y : x * y,
                              sequences=[x], non_sequences=[y])

f = th.function(inputs=[x, y],
                    outputs=output,
                    updates=updates)

In [ ]:
f(np.ones(10000),3)

In [ ]:
srng = T.shared_randomstreams.RandomStreams(seed=234)
rv_n = srng.normal((2,5), avg=0.1, std=1.0)

g = th.function([], rv_n)    #Not updating rv_n.rng
g()

In [ ]:
rv_n = srng.normal((2,5000), avg=1.0, std=1.0)

output, updates = th.scan(fn=lambda  rv_n, z : z + rv_n ,
                              sequences=[rv_n],
                              outputs_info=[z])

f = th.function(inputs=[z],
                    outputs=output,
                    updates=updates)
out = f(np.zeros((1,5000)))
print(out.shape)
print(out.mean(axis=-1))

In [ ]:
rv_n = srng.normal((2,5000), avg=0.0, std=1.0)
k = T.dscalar()
z = T.dvector('z')
output, updates = th.scan(fn=lambda  rv_n,k, z : z + rv_n +0*k,
                              sequences=[rv_n], non_sequences=[k], 
                              outputs_info=[z])

f = th.function(inputs=[k, z],
                    outputs=output,
                    updates=updates)
out = f(2.0, np.zeros((5000)))
print(out.shape)
print(out.mean(axis=-1))

In [ ]:
rv_n = srng.normal((40000,1000), avg=0.0, std=1.0)
k = T.dscalar('k')
z = T.dmatrix('z')
output, updates = th.scan(fn=lambda  rv_n,k, z : z + rv_n * k,
                              sequences=[rv_n], non_sequences=[k], 
                              outputs_info=[z])

f = th.function(inputs=[k, z],
                    outputs=output,
                    updates=updates)


In [ ]:
%%time
f(1, np.zeros((1,1000))).shape

In [6]:
import sys 
sys.setrecursionlimit(50000)


k = T.dscalar('k')
z = T.dmatrix('z')
r = z + k
for _ in range(1000):
    #rv_n = srng.normal(size=(1,1000), avg=0.0, std=1.0)
    r = r + k
f = th.function(inputs=[k, z], outputs=r)

In [16]:
%%timeit
f(1, np.zeros((1,1000))).shape

3.06 ms ± 35.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%%timeit
r = 0
k = np.zeros((1,1000))
r = r + k
for _ in range(1000):
    #rv_n = srng.normal(size=(1,1000), avg=0.0, std=1.0)
    r = 1 + k*0.1

2.39 ms ± 16.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%time
r = np.zeros(1000)
for i in range(40000):
# Step simulation
    r = r + np.random.normal(1000)
print(r.shape)

## Trying with loop